# Prompt tuning

In [2]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.utils.data import Dataset
import tqdm
import json
import re
from peft import PeftModel

In [4]:
cd drive/MyDrive/dialouge_summarization

/content/drive/MyDrive/dialouge_summarization


## 모델 불러오기

- Version 1. baseline 모델
- Version 2. baseline 모델 + fine-tuning

In [6]:
version = 2
model_id = 'MLP-KTLim/llama-3-Korean-Bllossom-8B'
device = 'cuda:0'
adapter_checkpoint_path = './resource/results/checkpoint-62/'
output = 'test_results.json'

In [7]:
## loading model ##
model = AutoModelForCausalLM.from_pretrained(
        model_id,
        torch_dtype=torch.bfloat16,
        device_map=device,
        # return_dict=True,
        low_cpu_mem_usage=True
    )

if version == 2:
    model = PeftModel.from_pretrained(model, adapter_checkpoint_path)
    model = model.merge_and_unload()
    model.to(dtype = torch.bfloat16)

model.eval()

## loading tokenizer ##
tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer.pad_token = tokenizer.eos_token
terminators = [tokenizer.eos_token_id,tokenizer.convert_tokens_to_ids("<|eot_id|>")]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## 예시 데이터 불러오기

- 10개의 샘플 데이터에 대해 prompt 튜닝시 모델의 출력이 어떻게 바뀌는지 확인

In [8]:
pre_prompt = '''You are a helpful AI assistant. Please answer the user's questions kindly.
당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.'''

In [9]:
class CustomDataset(Dataset):
    def __init__(self, fname, tokenizer):
        IGNORE_INDEX=-100
        self.inp = []
        self.label = []

        PROMPT = pre_prompt

        with open(fname, "r") as f:
            data = json.load(f)

        def make_chat(inp):
            chat = ["[Conversation]"]
            for cvt in inp['conversation']:
                speaker = cvt['speaker']
                utterance = cvt['utterance']
                chat.append(f"화자{speaker}: {utterance}")
            chat = "\n".join(chat)

            speakers = list(set(re.findall(r'SD\d{7}', chat)))

            question =  f'''[Question]\n위 {', '.join(inp['subject_keyword'])} 주제에 대한 대화를 다음과 같은 순서로 요약해주세요.
            \n 맨 첫번째 문장은 전반적인 요약이 담겨있어야 하고, 이어서 속에 {speakers[0]}가 말한 모든 내용을 여러 문장으로 요약하고, 마지막으로 Conversation 속에 {speakers[1]}가 말한 모든 내용을 여러 문장으로 요약해주세요.
            \n [{speakers[0]}의 말 요약]처럼 소제목을 사용하지 말고
            \n 단순하게 여러 문장을 '.' 기준으로 이어붙인 구조로 답변해주세요.''' # post_prompt
            chat = chat + "\n\n" + question

            return chat

        for example in data:
            chat = make_chat(example["input"])
            message = [
                {"role": "system", "content": PROMPT},
                {"role": "user", "content": chat},
            ]

            source = tokenizer.apply_chat_template(
                message,
                add_generation_prompt=True,
                return_tensors="pt",
            )

            target = example["output"]
            if target != "":
                target += tokenizer.eos_token
            target = tokenizer(target,
                      return_attention_mask=False,
                      add_special_tokens=False,
                      return_tensors="pt")
            target["input_ids"] = target["input_ids"].type(torch.int64)

            input_ids = torch.concat((source[0], target["input_ids"][0]))
            labels = torch.concat((torch.LongTensor([IGNORE_INDEX] * source[0].shape[0]), target["input_ids"][0]))
            self.inp.append(input_ids)
            self.label.append(labels)

    def __len__(self):
        return len(self.inp)

    def __getitem__(self, idx):
        return self.inp[idx]

In [10]:
dataset = CustomDataset("resource/data/일상대화요약_test.json", tokenizer)[:10]

## Inference

In [11]:
with open("resource/data/일상대화요약_test.json", "r") as f:
    result = json.load(f)

for idx in tqdm.tqdm(range(len(dataset))):
    inp = dataset[idx]
    outputs = model.generate(
        inp.to(device).unsqueeze(0),
        max_new_tokens=1024,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        no_repeat_ngram_size= 5, # prevent repeated sentences
        # num_beams = 4
    )

    result[idx]["output"] = tokenizer.decode(outputs[0][inp.shape[-1]:], skip_special_tokens=True)

with open(output, "w", encoding="utf-8") as f:
    f.write(json.dumps(result, ensure_ascii=False, indent=4))

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
100%|██████████| 10/10 [03:42<00:00, 22.25s/it]


## 이전 결과와 비교

In [12]:
prev_result_path = 'result.json'
post_result_path = 'test_results.json'

In [13]:
with open(prev_result_path, "r") as f:
    prev_result = json.load(f)

with open(post_result_path, "r") as f:
    post_result = json.load(f)


for idx in tqdm.tqdm(range(len(dataset))):
    prev_output = prev_result[idx]["output"]
    post_output = post_result[idx]["output"]
    print(f"\n [[Sample {idx} ]] \n\n <<prev_output>> \n {prev_output} \n\n <<post_output>> \n {post_output} \n\n\n")

100%|██████████| 10/10 [00:00<00:00, 5032.16it/s]


 [[Sample 0 ]] 

 <<prev_output>> 
 SD2000044는 여름에 팥빙수를 자주 먹으며, 특히 설빙의 초코 빙수를 좋아한다고 말했습니다. 초코와 딸기, 오레오 등 초코 관련 빙수도 좋아한다고 덧붙였습니다. SD2000045는 여름 과일 중 복숭아와 수박을 좋아하며, 여름을 기다리는 이유는 수박을 먹고 티브이를 보며 추억을 만드는 것이라고 말했습니다. 또한, 추운 날에는 아이스 음료를 좋아한다고 하였고, SD2000044는 추울 때 아이스크림을 먹으면 기분이 좋다고 말했습니다. 

 <<post_output>> 
 SD2000045는 여름에 과일을 많이 먹는 스텀일이라고 말했어요. 봄에서 여울 넘어갈 때 과일이 비싸지 않아서 그때는 복숭아와 수박을 많이 먹는다고 했어요. 여름을 기다리는 이유는 수박을 먹고 티비를 보면서 추억을 만들기 때문이라고 말했어요.

SD2000044는 여름에 먹은 음식에 대해 말했어요. 팥빙수를 많이 먹는다고 말했어요. 설빙에서 초코 빙수를 가장 좋아하는데 초코와 딸기, 오레오 같은 초코 관련 빙수도 좋아해요. 초코를 많이 좋아하는 편이라고 말했어요.
SD2000044가 여름에 팟빙수를 많이 안 먹는다고 말한 name1 님과 달리 팥빙수를 좋아하는 것 같아요. 




 [[Sample 1 ]] 

 <<prev_output>> 
 이 대화는 크리스마스, 추석, 설날에 대한 대화를 요약한 내용입니다.

**크리스마스**
- **name1 님**: 크리스마스에는 크리스마스 특유의 분위기와 캐럴이 좋아. 특별한 계획은 없지만 집에서 친구들과 놀거나 아르바이트를 하며 평소와 똑같이 보내지만 좋은 추억이 있다.
- **name2 님**: 크리스마스에는 캐럴과 따뜻한 분위기가 좋아. 크리스마스에는 특별한 계획은 없지만 집에서 놀거나 남포동에서 놀거나 아르바이트를 하며 평소와 똑같이 보내지만 좋은 추억이 있다.

**추석**
- **name1 님**: 어릴 때는 친척들이랑 만나서 재미있게 놀다가 헤어질 때가 무서워서 가고

## Full Inference

In [14]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [6]:
!python -m run.test \
    --output result.json \
    --model_id MLP-KTLim/llama-3-Korean-Bllossom-8B \
    --device cuda:0 \
    --adapter_checkpoint_path ./resource/results/checkpoint-62/

Loading checkpoint shards: 100% 4/4 [00:05<00:00,  1.47s/it]
tokenizer_config.json: 100% 51.1k/51.1k [00:00<00:00, 4.59MB/s]
tokenizer.json: 100% 9.09M/9.09M [00:00<00:00, 51.3MB/s]
special_tokens_map.json: 100% 444/444 [00:00<00:00, 3.29MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
  0% 0/408 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:567: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:572: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The att